In [1]:
import sys

from imports.ExtractContactCases import ExtractContactCases
from imports.EventArrayAugmention import *
import math
import numpy as np
from pprint import pprint

ex = ExtractContactCases(
    tactile_bag_dir='../data/bags/small3/',
    outdir='../data/small/contact_extraction6',
    n_init_events=7000,
)

pprint(ex.params)

{'N_examples': 5,
 'augmentations': [],
 'bag': '/home/hussain/me/projects/tactile/data/bags/small3',
 'case_span': 2660000000.0,
 'center': (180, 117),
 'circle_rad': 85,
 'delta_t': 75000000.0,
 'margin': -25000000.0,
 'min_n_events': 750,
 'n_init_events': 7000,
 'possible_angles': [0.15707963267948966],
 'theta': [0, 0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966],
 'train_prop': 0.75}


In [2]:
ex.extract()

extracting contact timestamps:   0%|          | 0/27660 [00:00<?, ?it/s]

extracting event arrays:   0%|          | 0/120 [00:00<?, ?it/s]

saving
96 120


AssertionError: 

In [2]:
from models.modules import MaxPooling, MaxPoolingX
import torch
import torch_geometric

from torch.nn import Linear
from torch.nn.functional import elu
from torch_geometric.nn.conv import SplineConv
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.transforms import Cartesian
from torch_geometric.data import Data, Batch
from torch_geometric.nn.pool import radius_graph, knn_graph

from models.modules import TransitionDown
from torch_geometric import seed_everything

seed_everything(0)

class GraphRes(torch.nn.Module):

    def __init__(self):
        super(GraphRes, self).__init__()
        dim = 3

        bias = False
        root_weight = False
        pooling_size=(16/346, 12/260)

        # Set dataset specific hyper-parameters.
        kernel_size = 2
        n = [1, 8, 16, 16, 16, 32, 32, 32, 32]
        pooling_outputs = 16
        #kernel_size = 8
        #n = [1, 16, 32, 32, 32, 128, 128, 128]
        #pooling_outputs = 128

        self.conv1 = SplineConv(n[0], n[1], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm1 = BatchNorm(in_channels=n[1])
        self.td1 = TransitionDown(8, 8, ratio=0.25)
        
        self.conv2 = SplineConv(n[1], n[2], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm2 = BatchNorm(in_channels=n[2])

        self.pool7 = MaxPoolingX(0.25, size=16)
        self.fc = Linear(pooling_outputs * 16, out_features=2, bias=bias)

    def forward(self, data: torch_geometric.data.Batch) -> torch.Tensor:
        data.x = elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm1(data.x)
        x, pos, batch = self.td1(data.x, data.pos, data.batch)
        
        edge_index = radius_graph(pos, r=0.05, max_num_neighbors=16)
        data = Batch(x=x, pos=pos, batch=batch, edge_index=edge_index, y=data.y)
        data = Cartesian()(data)
        data.x = elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm2(data.x)

        x = self.pool7(data.x, pos=data.pos[:, :2], batch=data.batch)
        x = x.view(-1, self.fc.in_features)
        return self.fc(x)

    
    
class GraphRes(torch.nn.Module):

    def __init__(self):
        super(GraphRes, self).__init__()
        dim = 3

        bias = False
        root_weight = False
        pooling_size=(16/346, 12/260)

        # Set dataset specific hyper-parameters.
        kernel_size = 2
        n = [1, 8, 16, 16, 16, 32, 32, 32, 32]
        pooling_outputs = 32
        #kernel_size = 8
        #n = [1, 16, 32, 32, 32, 128, 128, 128]
        #pooling_outputs = 128

        self.conv1 = SplineConv(n[0], n[1], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm1 = BatchNorm(in_channels=n[1])
        self.conv2 = SplineConv(n[1], n[2], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm2 = BatchNorm(in_channels=n[2])
        self.td2 = TransitionDown(n[2], n[2], 0.5)
        
        self.conv3 = SplineConv(n[2], n[3], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm3 = BatchNorm(in_channels=n[3])
        self.conv4 = SplineConv(n[3], n[4], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm4 = BatchNorm(in_channels=n[4])

        self.conv5 = SplineConv(n[4], n[5], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm5 = BatchNorm(in_channels=n[5])
        self.pool5 = MaxPooling(pooling_size, transform=Cartesian(norm=True, cat=False))

        self.conv6 = SplineConv(n[5], n[6], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm6 = BatchNorm(in_channels=n[6])
        self.conv7 = SplineConv(n[6], n[7], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm7 = BatchNorm(in_channels=n[7])

        self.pool7 = MaxPoolingX(0.25, size=16)
        self.fc = Linear(pooling_outputs * 16, out_features=2, bias=bias)

    def forward(self, data: torch_geometric.data.Batch) -> torch.Tensor:
        data.x = elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm1(data.x)
        data.x = elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm2(data.x)
        
        x, pos, batch = self.td2(data.x, data.pos, data.batch)
        edge_index = radius_graph(pos, r=0.05, max_num_neighbors=16)
        data = Batch(x=x, pos=pos, batch=batch, edge_index=edge_index, y=data.y)
        data = Cartesian()(data)

        x_sc = data.x.clone()
        data.x = elu(self.conv3(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm3(data.x)
        data.x = elu(self.conv4(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm4(data.x)
        data.x = data.x + x_sc

        data.x = elu(self.conv5(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm5(data.x)
        data = self.pool5(data.x, pos=data.pos, batch=data.batch, edge_index=data.edge_index, return_data_obj=True)

        x_sc = data.x.clone()
        data.x = elu(self.conv6(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm6(data.x)
        data.x = elu(self.conv7(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm7(data.x)
        data.x = data.x + x_sc

        x = self.pool7(data.x, pos=data.pos[:, :2], batch=data.batch)
        x = x.view(-1, self.fc.in_features)
        return self.fc(x)

In [4]:
model = GraphRes()
model.load_state_dict(torch.load('../results/exp3/state_dict', map_location=torch.device('cpu') ))

<All keys matched successfully>

In [5]:
from imports.TactileDataset import TactileDataset
from torch_geometric.loader import DataLoader

train_1 = TactileDataset('../data/small/contact_extraction1/train/')
val_loader = DataLoader(train_1)

{'bag': '/home/hussain/me/projects/tactile/data/bags/small2', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_events': -1, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 5, 'possible_angles': [0.15707963267948966], 'theta': [0, 0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966], 'augmentations': [], 'node_features': 'pol'}


In [6]:
sampe = train_1[0]
sampe

Data(x=[14727, 1], edge_index=[2, 225365], y=[1, 2], pos=[14727, 3], edge_attr=[225365, 3])

In [7]:
model(sampe)

/home/hussain/miniconda3/envs/tactile/lib/python3.8/site-packages/torch_geometric/nn/conv/spline_conv.py:133: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'max'

In [6]:
from torch_geometric.transforms import FixedPoints

fp = FixedPoints(1000)

In [8]:
fp(sampe)

Data(x=[1000, 1], edge_index=[2, 225365], y=[1, 2], pos=[1000, 3], edge_attr=[225365, 3])

In [11]:
try:
    import rosbag
    print(1)
except ImportError:
    pass

1


In [9]:
import p

ModuleNotFoundError: No module named 'p'

In [4]:
row, col = train_1[0].edge_index

In [7]:
t = train_1[0].pos[:, 2]

In [9]:
t[row] > t[col]

tensor([ True,  True,  True,  ..., False, False, False])

In [12]:
from imports.TrainModel import TrainModel

#from models.modules import GraphRes

#!rm -rf ../data/small/contact_extraction1/{test,train,val}/processed/*.pt
tm = TrainModel(
    '../data/small/contact_extraction4/', 
    GraphRes().cuda(), 
    lr=0.01, 
    features='pol', 
    batch=4, 
    n_epochs=300, 
    experiment_name='exp5', 
    desc='aegnn model small on raidal data 24 iters. bigger val',
    merge_test_val=True
)

{'bag': '/home/hussain/me/projects/tactile/data/bags/small3', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_events': -1, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 5, 'possible_angles': [0.15707963267948966], 'theta': [0, 0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966], 'augmentations': [], 'node_features': 'pol'}
{'bag': '/home/hussain/me/projects/tactile/data/bags/small3', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_events': -1, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 5, 'possible_angles': [0.15707963267948966], 'theta': [0, 0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966], 'augmentations': [], 'node_features': 'pol'}
{'bag': '/home/hussain/me/projects/tactile/data/bags/small3', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_events'

/home/hussain/me/projects/tactile/src/imports/TrainModel.py:39: UserWarning: experiment /home/hussain/me/projects/tactile/results/exp4 already exists. Press enter to proceed overwriting.
  warnings.warn(f'experiment {str(path)} already exists. Press enter to proceed overwriting.')


In [1]:
tm.val_loader = val_loader

NameError: name 'val_loader' is not defined

In [7]:
tm.train()

training:   0%|          | 0/300 [00:00<?, ?epoch/s]

  0%|          | 0/23 [00:00<?, ?batch/s]

/home/hussain/me/projects/tactile/src/imports/TrainModel.py:105: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():


logging


  0%|          | 0/23 [00:00<?, ?batch/s]

...

logging


  0%|          | 0/23 [00:00<?, ?batch/s]

logging


  0%|          | 0/23 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [8]:
torch.save(tm.model.state_dict, '../results/exp4/state_dict')

In [5]:
loss_func = torch.nn.L1Loss()
import numpy as np
out=np.degrees(([loss_func(pred[i].cpu(), true[i].cpu()).detach().item() for i in range(len(pred))]))

In [6]:
out

array([1.01537614, 2.89507685, 2.19135435, 1.39343607, 0.5789326 ,
       2.85330982, 1.58974723, 1.32754331, 1.30032949, 1.18831343,
       1.63290528, 2.42328367, 0.49593769, 1.8088215 , 2.41923081,
       1.75745154, 2.79286905, 2.81849314, 1.35552617, 1.06889152,
       1.72093574, 0.46053874, 2.26759186, 1.86306579, 3.05377993,
       0.84769456, 1.0608017 , 3.25267354, 2.07939474, 1.90800182])

In [7]:
np.mean(out)

1.7807102689669057